# Exercise 1: Confidence Intervals - NHANES


This exercise, we are going to practice on how to load data, clean/manipulate a dataset, and construct a confidence interval for the difference between two population proportions and means.

We will use the 2015-2016 wave of the NHANES data for our analysis.

For our population proportions, we will analyze the difference of proportion between female and male smokers.  The column that specifies smoker and non-smoker is "SMQ020" in our dataset.

For our population means, we will analyze the difference of mean of body mass index within our female and male populations.  The column that includes the body mass index value is "BMXBMI".

Additionally, the gender is specified in the column "RIAGENDR".

In [1]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [2]:
url = "nhanes_2015_2016.csv"
da = pd.read_csv(url)
da

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,124.0,64.0,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,140.0,88.0,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,132.0,44.0,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,134.0,68.0,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5730,93695,2.0,2.0,NaN,1,2,76,3,1.0,3.0,...,112.0,46.0,59.1,165.8,21.5,38.2,37.0,29.5,95.0,2.0
5731,93696,2.0,2.0,NaN,2,1,26,3,1.0,5.0,...,116.0,76.0,112.1,182.2,33.8,43.4,41.8,42.3,110.2,2.0
5732,93697,1.0,NaN,1.0,1,2,80,3,1.0,4.0,...,146.0,58.0,71.7,152.2,31.0,31.3,37.5,28.8,NaN,2.0
5733,93700,NaN,NaN,NaN,1,1,35,3,2.0,1.0,...,106.0,66.0,78.2,173.3,26.0,40.3,37.5,30.6,98.9,2.0


### Investigating and Cleaning Data

Create a new column named 'SMQ020x' and store data from column 'SMQ020' with following replacements:

- 1 to "Yes"
- 2 to "No"
- 7 to NaN
- 9 to NaN

In [3]:
def smq(x):
    if x == 1:
        return "Yes"
    elif x == 2:
        return "No"
    else: 
        return "NaN"

da['SMQ020x'] = da['SMQ020'].apply(smq)
da

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210,SMQ020x
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,64.0,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0,Yes
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,88.0,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN,Yes
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,44.0,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0,Yes
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,68.0,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0,No
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5730,93695,2.0,2.0,NaN,1,2,76,3,1.0,3.0,...,46.0,59.1,165.8,21.5,38.2,37.0,29.5,95.0,2.0,Yes
5731,93696,2.0,2.0,NaN,2,1,26,3,1.0,5.0,...,76.0,112.1,182.2,33.8,43.4,41.8,42.3,110.2,2.0,No
5732,93697,1.0,NaN,1.0,1,2,80,3,1.0,4.0,...,58.0,71.7,152.2,31.0,31.3,37.5,28.8,NaN,2.0,Yes
5733,93700,NaN,NaN,NaN,1,1,35,3,2.0,1.0,...,66.0,78.2,173.3,26.0,40.3,37.5,30.6,98.9,2.0,Yes


Create a new column named 'RIAGENDRx' and store data from column 'RIAGENDR' with following replacements:

- 1 to "Male"
- 2 to "Female"

In [4]:
def sex(x):
    if x == 1:
        return "Male"
    elif x == 2:
        return "Female"
    else: 
        return "NaN"

da['RIAGENDRx'] = da['RIAGENDR'].apply(sex)

Drop all NAs from both `SMQ020x` & `RIAGENDRx` and store into a new dataframe named 'dx'. Plot the following crosstab using `pd.crosstab` library.

In [5]:
dx = da.drop(da[da['SMQ020x'] == 'NaN'].index)
dx.drop(dx[dx['RIAGENDRx'] == 'NaN'].index, inplace=True)
dx = dx[['SMQ020x', 'RIAGENDRx']]
pd.crosstab(dx['SMQ020x'], dx['RIAGENDRx'])

RIAGENDRx,Female,Male
SMQ020x,,
No,2066,1340
Yes,906,1413


Replace `dx['SMQ020x']` "Yes" to 1 and "No" to 0.

In [6]:
def yn(x):
    if x == "Yes":
        return 1
    elif x == "No":
        return 0
    else: 
        return "NaN"

dx['SMQ020x'] = dx['SMQ020x'].apply(yn)
dx

,SMQ020x,RIAGENDRx
0,1,Male
1,1,Male
2,1,Male
3,0,Female
4,0,Female
...,...,...
5730,1,Female
5731,0,Male
5732,1,Female
5733,1,Male


In [7]:
dx.groupby(['RIAGENDRx'])['SMQ020x'].value_counts()

RIAGENDRx  SMQ020x
Female     0          2066
           1           906
Male       1          1413
           0          1340
Name: SMQ020x, dtype: int64

Calculate the 'mean' and 'size' and store into a new dataframe called `dz`

In [8]:
data = { "Mean" : [dx[dx['RIAGENDRx'] == "Male"]['SMQ020x'].mean(),dx[dx['RIAGENDRx'] == "Female"]['SMQ020x'].mean() ],
        "Size" : [dx[dx['RIAGENDRx'] == "Male"]['SMQ020x'].count(),dx[dx['RIAGENDRx'] == "Female"]['SMQ020x'].count() ],
        "Sex" : ['Male', 'Female']
    }
dz = pd.DataFrame(data, columns=['Mean', 'Size','Sex'])
dz.set_index('Sex',inplace=True)
dz

,Mean,Size
Sex,,
Male,0.513258,2753
Female,0.304845,2972


### Constructing Confidence Intervals

Now that we have the population proportions of male and female smokers, we can begin to calculate confidence intervals.  From lecture, we know that the equation is as follows:

$$Best\ Estimate \pm Margin\ of\ Error$$

Where the *Best Estimate* is the **observed population proportion or mean** from the sample and the *Margin of Error* is the **t-multiplier**.

The equation to create a 95% confidence interval can also be shown as:

$$Population\ Proportion\ or\ Mean\ \pm (t-multiplier *\ Standard\ Error)$$

The Standard Error is calculated differenly for population proportion and mean:

$$Standard\ Error \ for\ Population\ Proportion = \sqrt{\frac{Population\ Proportion * (1 - Population\ Proportion)}{Number\ Of\ Observations}}$$

$$Standard\ Error \ for\ Mean = \frac{Standard\ Deviation}{\sqrt{Number\ Of\ Observations}}$$

Lastly, the standard error for difference of population proportions and means is:

$$Standard\ Error\ for\ Difference\ of\ Two\ Population\ Proportions\ Or\ Means = \sqrt{SE_{Proportion\ 1}^2 + SE_{Proportion\ 2} ^2}$$

#### Difference of Two Population Proportions

REFER PAGE 41 in pdf


Calculate the standard error for female

In [9]:
import math
fem = dx[(dx['RIAGENDRx'] == "Female") & (dx['SMQ020x'] == 1)].value_counts().sum()
p1 = fem / 2972
sef = math.sqrt((p1*(1-p1))/2972)
sef

0.008444152146214435

Calculate the standard error for male

In [10]:
import math
man = dx[(dx['RIAGENDRx'] == "Male") & (dx['SMQ020x'] == 1)].value_counts().sum()
p2 = man / 2753
sem = math.sqrt((p2*(1-p2))/2753)
sem

0.009526078653689868

Calculate the difference between these two Standard Errors

In [11]:
diff = np.sqrt(sem**2 + sef**2)
diff

0.012729881381407434

Calculate the confidence Interval

In [12]:
d = dz.iloc[0][0] - dz.iloc[1][0]
print(d-2*diff, d+2*diff)

0.18295327887682067 0.2338728044024504


#### Difference of Two Population Means

Now we look into the differences between 2 population means

In [20]:
pop = da.groupby(['RIAGENDRx']).agg({"BMXBMI":[np.mean,np.std,np.size]})
pop

BMXBMI                  
                mean       std    size
RIAGENDRx                             
Female     29.939946  7.753319  2976.0
Male       28.778072  6.252568  2759.0

Calculate the Standard Error for Mean for both female and male

In [16]:
fem = 7.753/np.sqrt(dz.iloc[1]["Size"])
man = 6.253 / np.sqrt(dz.iloc[0]['Size'])
print(fem,man)

0.14221499207323504 0.11917504456014921


Calculate the difference between 2 Standard Error for Mean

In [17]:
error = np.sqrt(fem**2 + man**2)
error

0.18554728566137488

The difference between two means for male and female

In [25]:
mean = pop.iloc[0][0] - pop.iloc[1][0]
mean

1.1618735403270115

Calculate the confidence interval between two population means

In [26]:
print(mean-2*error,mean+2*error)

0.7907789690042617 1.5329681116497613
